In [1]:
# Load packages
import os
import pandas as pd 
import geopandas as gpd 
from shapely.geometry import Point
import numpy as np 

In [2]:
# Load osm_building Features and calculate polygon centroids
home_path = os.getenv('HOME')
osm_building_shp = home_path+'/GeoData/OSM/algeria-latest-free.shp/gis_osm_buildings_a_free_1.shp'
osm_building_gdf = gpd.read_file(osm_building_shp)
osm_building_gdf.head()

,osm_id,code,fclass,name,type,geometry
0,23453590,1500,building,None,None,"POLYGON ((3.28630 36.72337, 3.28680 36.72384, ..."
1,23453621,1500,building,None,industrial,"POLYGON ((3.28839 36.72555, 3.28945 36.72653, ..."
2,23453649,1500,building,None,industrial,"POLYGON ((3.29652 36.73153, 3.29695 36.73203, ..."
3,26224008,1500,building,Hilton Alger,hotel,"POLYGON ((3.15595 36.73836, 3.15634 36.73856, ..."
4,26224288,1500,building,Crystal Lounge,None,"POLYGON ((3.15561 36.73811, 3.15583 36.73823, ..."


In [3]:
# Set the correct CRS and reproject
osm_building_gdf = osm_building_gdf.set_crs("EPSG:4326")
osm_building_gdf = osm_building_gdf.to_crs("EPSG:3857")

osm_building_gdf.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World - 85°S to 85°N
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [4]:
# Load the targeted dataset
pois_path = '../dz-datasets/notaries/'
pois = gpd.read_file(pois_path + 'pois_checked_in_out_wilayas.geojson')
pois.head()

,num,name,address,code_wil,court,tel,status,lat,lon,to_check,geometry
0,1,BARIK BOUDJEMAA,CITE 103 LOGTS BP 575 Adrar,1,Adrar,049 96 53 89,physical,28.017440,-0.264250,False,POINT (-0.26425 28.01744)
1,2,BENHADJ AHMED,CITE DES 40 LOGTS LOCAL 25 Adrar,1,Adrar,049 96 67 26,physical,28.017440,-0.264250,False,POINT (-0.26425 28.01744)
2,3,AZZOU ABD ERRAHMANE,Rue Amrad Cité OULED ANKAL Adrar,1,Adrar,049 96 53 19,physical,36.102009,2.712287,True,POINT (2.71229 36.10201)
3,4,MIMOUNI ABDELKADER,"Rue Mokadem El Arbi, lot 30 Group 191 Adrar",1,Adrar,0665 01 20 18,physical,28.017440,-0.264250,False,POINT (-0.26425 28.01744)
4,5,OULED ALI AHMED,Cité 60 Logements Timimoun Adrar,1,Timimoun,0555 40 62 94,physical,35.261165,6.368815,True,POINT (6.36881 35.26116)


In [5]:
# Define source CRS to geographic and project to a Pseudo-Mercator
# Useful for spatial analysis (buffer, intersect, ...)

pois = pois.set_crs("EPSG:4326")
pois = pois.to_crs("EPSG:3857")
pois.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World - 85°S to 85°N
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [6]:
# Replace DataFrame geometries with buffers and write it to a file
pois['buffer'] = pois.geometry.buffer(500)
buffers = pois[['num', 'buffer']]
buffers = buffers.rename(columns={'buffer': 'geometry'})
buffers = gpd.GeoDataFrame(buffers)
buffers = buffers.set_crs("EPSG:3857")
#gpd.GeoDataFrame(buffers).to_file(pois_path + "buffers.geojson", driver='GeoJSON')
buffers.head()

,num,geometry
0,1,"POLYGON ((-28916.142 3251172.790, -28918.550 3..."
1,2,"POLYGON ((-28916.142 3251172.790, -28918.550 3..."
2,3,"POLYGON ((302430.419 4314666.719, 302428.011 4..."
3,4,"POLYGON ((-28916.142 3251172.790, -28918.550 3..."
4,5,"POLYGON ((709473.243 4199429.307, 709470.835 4..."


In [7]:
# Search for buildings that intersect the locations buffers
res_intersection = gpd.overlay(buffers, osm_building_gdf, how='intersection')
res_intersection.count()

num         88120
osm_id      88120
code        88120
fclass      88120
name         8047
type        21841
geometry    88120
dtype: int64

In [10]:
res_intersection.head()

,num,osm_id,code,fclass,name,type,geometry
0,3,531250778,1500,building,None,residential,"POLYGON ((302264.440 4315037.976, 302283.972 4..."
1,6,745144709,1500,building,None,None,"POLYGON ((26441.936 3409207.201, 26453.428 340..."
2,7,296725259,1500,building,APC,office,"POLYGON ((120045.570 3120061.364, 120077.663 3..."
3,7,296922214,1500,building,Maison de justice,detached,"POLYGON ((120320.351 3120156.905, 120329.635 3..."
4,7,296926257,1500,building,None,commercial,"POLYGON ((120333.331 3119914.412, 120342.470 3..."


In [11]:
# Write intersected building into a GeoJSON file
res = res_intersection[['num', 'geometry']]
#res.to_file(pois_path + "intersection.geojson", driver='GeoJSON')
res.head()

,num,geometry
0,3,"POLYGON ((302264.440 4315037.976, 302283.972 4..."
1,6,"POLYGON ((26441.936 3409207.201, 26453.428 340..."
2,7,"POLYGON ((120045.570 3120061.364, 120077.663 3..."
3,7,"POLYGON ((120320.351 3120156.905, 120329.635 3..."
4,7,"POLYGON ((120333.331 3119914.412, 120342.470 3..."


In [12]:
# Group results of intersection and add a to_check column (if intersect then doesn't need to check)
res_grouped = res [['num']]
res_grouped = res_grouped.groupby('num').count()
res_grouped['to_check_osm'] = False
res_grouped.count()

to_check_osm    818
dtype: int64

In [13]:
# Merge with DataFrame and set geometry to centroids
pois_gdf = pois.merge(res_grouped, left_on='num', right_on='num', how='left')
pois_gdf.drop(columns=['buffer'], inplace=True)
pois_gdf.count()

num             892
name            892
address         892
code_wil        892
court           335
tel             296
status          310
lat             883
lon             883
to_check        892
geometry        892
to_check_osm    818
dtype: int64

In [14]:
# Update to_check column :if doesn't intersect then we need to check it manually
pois_gdf['to_check_osm'] = pois_gdf.apply(lambda row: row.to_check_osm != False, axis=1)
pois_gdf[pois_gdf['to_check_osm'] != False]

,num,name,address,code_wil,court,tel,status,lat,lon,to_check,geometry,to_check_osm
0,1,BARIK BOUDJEMAA,CITE 103 LOGTS BP 575 Adrar,1,Adrar,049 96 53 89,physical,28.017440,-0.264250,False,POINT (-29416.142 3251172.790),True
1,2,BENHADJ AHMED,CITE DES 40 LOGTS LOCAL 25 Adrar,1,Adrar,049 96 67 26,physical,28.017440,-0.264250,False,POINT (-29416.142 3251172.790),True
3,4,MIMOUNI ABDELKADER,"Rue Mokadem El Arbi, lot 30 Group 191 Adrar",1,Adrar,0665 01 20 18,physical,28.017440,-0.264250,False,POINT (-29416.142 3251172.790),True
4,5,OULED ALI AHMED,Cité 60 Logements Timimoun Adrar,1,Timimoun,0555 40 62 94,physical,35.261165,6.368815,True,POINT (708973.243 4199429.307),True
16,13,SAMET BOUHAYEK DJILLALI,26 RUE EL MOUSSAAD AZZOUN Chlef,2,None,None,None,32.177101,35.055957,True,POINT (3902411.283 3786580.402),True
...,...,...,...,...,...,...,...,...,...,...,...,...
877,883,ACHOUR NOUREDDINE,Batiement 07 Tazghat El Jilali Oued Rhiou Rel...,48,Oued Rhiou,046 78 62 73 / 0773 23 61 72,physical,35.977005,0.924750,False,POINT (102942.710 4297457.729),True
878,885,KHATHIR ABEDELLAH,RUE BEN KOUDIA MAAMAR Oued Rhiou Relizane,48,Oued Rhiou*,None,None,35.977005,0.924750,False,POINT (102942.710 4297457.729),True
880,882,HENNI ABD ERRAHMANE,Rue Tazghat El Jilali Oued Rhiou Relizane,48,Oued Rhiou,046 78 68 42 / 0550 27 25 40,physical,35.977005,0.924750,False,POINT (102942.710 4297457.729),True
881,881,AZIZI KHATHIR,RUE BEN KAHLA MENOUER Oued Rhiou Relizane,48,Oued Rhiou,046 78 76 78 / 0698 81 48 35,physical,35.977005,0.924750,False,POINT (102942.710 4297457.729),True


In [15]:
final_gdf = pois_gdf
final_gdf['to_check'] = (pois_gdf['to_check'] == True) | (pois_gdf['to_check_osm'] == True)   
final_gdf[final_gdf.to_check == True]

,num,name,address,code_wil,court,tel,status,lat,lon,to_check,geometry,to_check_osm
0,1,BARIK BOUDJEMAA,CITE 103 LOGTS BP 575 Adrar,1,Adrar,049 96 53 89,physical,28.017440,-0.264250,True,POINT (-29416.142 3251172.790),True
1,2,BENHADJ AHMED,CITE DES 40 LOGTS LOCAL 25 Adrar,1,Adrar,049 96 67 26,physical,28.017440,-0.264250,True,POINT (-29416.142 3251172.790),True
2,3,AZZOU ABD ERRAHMANE,Rue Amrad Cité OULED ANKAL Adrar,1,Adrar,049 96 53 19,physical,36.102009,2.712287,True,POINT (301930.419 4314666.719),False
3,4,MIMOUNI ABDELKADER,"Rue Mokadem El Arbi, lot 30 Group 191 Adrar",1,Adrar,0665 01 20 18,physical,28.017440,-0.264250,True,POINT (-29416.142 3251172.790),True
4,5,OULED ALI AHMED,Cité 60 Logements Timimoun Adrar,1,Timimoun,0555 40 62 94,physical,35.261165,6.368815,True,POINT (708973.243 4199429.307),True
...,...,...,...,...,...,...,...,...,...,...,...,...
878,885,KHATHIR ABEDELLAH,RUE BEN KOUDIA MAAMAR Oued Rhiou Relizane,48,Oued Rhiou*,None,None,35.977005,0.924750,True,POINT (102942.710 4297457.729),True
880,882,HENNI ABD ERRAHMANE,Rue Tazghat El Jilali Oued Rhiou Relizane,48,Oued Rhiou,046 78 68 42 / 0550 27 25 40,physical,35.977005,0.924750,True,POINT (102942.710 4297457.729),True
881,881,AZIZI KHATHIR,RUE BEN KAHLA MENOUER Oued Rhiou Relizane,48,Oued Rhiou,046 78 76 78 / 0698 81 48 35,physical,35.977005,0.924750,True,POINT (102942.710 4297457.729),True
890,886,ATTOU KADOUR,33 Cité 50 Logements Mazouna Relizane,48,Mazouna,046 94 81 76,physical,36.126470,0.890610,True,POINT (99142.274 4318037.432),True


In [16]:
# Drop unecessary columns
final_gdf = final_gdf.drop(columns=['to_check_osm'], axis=1)

In [17]:
# Reproject to geograĥic and Write to a GeoPackage file
final_gdf = final_gdf.set_crs("EPSG:3857")
final_gdf = final_gdf.to_crs("EPSG:4326")
final_gdf.to_file(pois_path + "pois_to_check.geojson", driver='GeoJSON')